In [1]:
import pandas as pd
import numpy as np

In [2]:
url = ('https://raw.githubusercontent.com/michelbierlaire/mooc-discrete-choice/master/'
       'syntheticPopulationWithChoice.zip')
population = pd.read_csv(url)

In [3]:
populationSize = population.shape[0]
sampleSize = 1000

# Simple random sample

## Calculation of the sampling probabilities

Simple random sample assigns the same probability to be sampled to each individual. We store it in the column <code>SRS</code>.

In [4]:
population['SRS'] = sampleSize / populationSize

We use the sample function of Pandas.

In [5]:
srsData = population.sample(n=1000)
srsData.shape

(1000, 18)

In [6]:
srsData.to_csv('srsData.csv', index=False)

# Sampling function

We first define a generic function for stratified sampling. It takes as arguments: 

- the name of the sampling scheme, that is used to create new columns in the database,
- the mask identifying the entries in the database corresponding with each group,
- the target shares for each group in the sample (the length of the list must be the number of groups).

The function returns $W$, the share of each group in the population, and the sample.

If the name of the sampling scheme is SS, say, two columns are added to the database: 

- the column <code>SS</code> contains the sampling probability of each individual,
- the column <code>SSGroup</code> contains the ID of the group of each individual.

Note the function <code>sampleStratum</code> that takes a stratum as argument and samples from it. The statement <code>x[groupname].mean()</code> retrieves the group ID from the column <code>SSGRoup</code>, so that the requested group size in the sample can be obtained. Two special cases need to be addressed: 

- if the requested size is zero, the function returns <code>None</code>,
- if the requested size exceeds the number of individuals actually present in this group in the population, the whole group is returned.

In [7]:
def sample(name, mask, H):
    # name: name of the sampling scheme
    # mask: list of masks identifying the strata
    # H: target shares for each stratum
    groupname = f'{name}Group'
    # Calculate the share of each group in the population
    W = [None] * len(mask)
    for i, m in enumerate(mask):
        W[i] = population[m].shape[0] / populationSize
        population.loc[m, groupname] = i
    # Sampling probabilities
    for s in range(4):
        population.loc[mask[s], name] = (
            H[s] * sampleSize / (W[s] * populationSize)
        )
    # Sampling
    groupsize = np.array(H) * sampleSize

    def sampleStratum(x):
        # The statement 'int(x[groupname].mean())' retrieves the index of
        # the group.
        size = int(groupsize[int(x[groupname].mean())])
        print(f'Sample {size} out of {x.shape[0]}')
        if size == 0:
            return None
        if size > x.shape[0]:
            print(
                'Warning: not enough individual in stratum '
                'to reach the requirements'
            )
            return x
        return x.sample(n=size)

    return (
        W,
        (
            population.groupby(groupname)
            .apply(sampleStratum)
            .reset_index(drop=True)
        ),
    )


# Exogenously stratified Sample

The strata are defined based on trip purpose and car availability. As the choice is not involved in the definition of strata, the stratified sampling is *exogenous*.

In [8]:
mask = [None]*4

Group 0: TripPurpose = work (1), CarAvail = yes (1) 

In [9]:
mask[0] = (population['TripPurpose'] == 1) & (population['CarAvail'] == 1)

Group 1: TripPurpose = work (1), CarAvail = no (not 1) 

In [10]:
mask[1] = (population['TripPurpose'] == 1) & (population['CarAvail'] != 1)

Group 2: TripPurpose = other (not 1), CarAvail = yes (1) 

In [11]:
mask[2] = (population['TripPurpose'] != 1) & (population['CarAvail'] == 1)

Group 3: TripPurpose = other (not 1), CarAvail = no (not 1) 

In [12]:
mask[3] = (population['TripPurpose'] != 1) & (population['CarAvail'] != 1)

Target shares in the sample: 25% for each stratum

In [13]:
H = [0.25]*4

Sampling

In [14]:
W, sampleXSS = sample('XSS', mask, H)

Sample 250 out of 322501
Sample 250 out of 21023
Sample 250 out of 626213
Sample 250 out of 30263


In [15]:
W

[0.322501, 0.021023, 0.626213, 0.030263]

Checksum: they should add up to one.

In [16]:
sum(W)

1.0

We verify the sample size

In [17]:
sampleXSS.shape[0]

1000

We verify the share of each group in the sample

In [18]:
actualShares = list(sampleXSS.groupby('XSSGroup').size() / sampleSize)
actualShares

[0.25, 0.25, 0.25, 0.25]

We save the sample on file

In [19]:
sampleXSS.to_csv('sampleXSS.csv', index=False)

## Endogenously stratified Sample 1

The strata are defined based on car availability and the chosen alternative.  As the choice is involved in the definition of strata, the stratified sampling is *endogenous*.

In [20]:
mask = [None] * 6

Group 0: CarAvail = yes (1), Choice = public transportation (0) 

In [21]:
mask[0] = (population['CarAvail'] == 1) & (population['Choice'] == 0)

Group 1: CarAvail = yes (1), Choice = car (1) 

In [22]:
mask[1] = (population['CarAvail'] == 1) & (population['Choice'] == 1)

Group 2: CarAvail = yes (1), Choice = slow mode (2) 

In [23]:
mask[2] = (population['CarAvail'] == 1) & (population['Choice'] == 2)

Group 3: CarAvail = no (not 1), Choice = public transportation (0) 

In [24]:
mask[3] = (population['CarAvail'] != 1) & (population['Choice'] == 0)

Group 4: CarAvail = no (not 1), Choice = car (1) 

In [25]:
mask[4] = (population['CarAvail'] != 1) & (population['Choice'] == 1)

Note: there is no individual in group 4. Indeed, if no car is available, car cannot be chosen.

Group 5: CarAvail = no (not 1), Choice = slow mode (2) 

In [26]:
mask[5] = (population['CarAvail'] != 1) & (population['Choice'] == 2)

Target shares in the sample: there are 5 non empty strata, taking 20% each

In [27]:
H = [0.2, 0.2, 0.2, 0.2, 0.0, 0.2]

In [28]:
W, sampleESS = sample('ESS', mask, H)

Sample 200 out of 234751
Sample 200 out of 649256
Sample 200 out of 64707
Sample 200 out of 50920
Sample 200 out of 366


In [29]:
W

[0.234751, 0.649256, 0.064707, 0.05092, 0.0, 0.000366]

Checksum: they should add up to one.

In [30]:
sum(W)

0.9999999999999999

We verify the sample size

In [31]:
sampleESS.shape[0]

1000

We verify the share of each group in the sample

In [32]:
actualShares = list(sampleESS.groupby('ESSGroup').size() / sampleSize)
actualShares

[0.2, 0.2, 0.2, 0.2, 0.2]

We save the sample on file

In [33]:
sampleESS.to_csv('sampleESS.csv', index=False)

## Endogenously stratified Sample 2

The strata are defined based on the trip purpose, the language and the chosen alternative.   As the choice is involved in the definition of strata, the stratified sampling is *endogenous*.

In [34]:
mask = [None] * 12

Group 0: CarAvail = yes (1), language = French (1), Choice = public transportation (0) 

In [35]:
mask[0] = (population['CarAvail'] == 1) & (population['LangCode'] == 1) & (population['Choice'] == 0)

Group 1: CarAvail = yes (1), language = French (1), Choice = car (1) 

In [36]:
mask[1] = (population['CarAvail'] == 1) & (population['LangCode'] == 1) & (population['Choice'] == 1)

Group 2: CarAvail = yes (1), language = French (1), Choice = slow mode (2) 

In [37]:
mask[2] = (population['CarAvail'] == 1) & (population['LangCode'] == 1) & (population['Choice'] == 2)

Group 3: CarAvail = no (not 1), language = French (1), Choice = public transportation (0) 

In [38]:
mask[3] = (population['CarAvail'] != 1) & (population['LangCode'] == 1) & (population['Choice'] == 0)

Group 4: CarAvail = no (not 1), language = French (1), Choice = car (1) 

In [39]:
mask[4] = (population['CarAvail'] != 3) & (population['LangCode'] == 1) & (population['Choice'] == 1)

There is  no individual in group 4. Indeed, if no car is available, car cannot be chosen.

Group 5: CarAvail = no (not 1), language = French (1), Choice = slow mode (2) 

In [40]:
mask[5] = (population['CarAvail'] != 1) & (population['LangCode'] == 1) & (population['Choice'] == 2)

Group 6: CarAvail = yes (1), language = German (2), Choice = public transportation (0) 

In [41]:
mask[6] = (population['CarAvail'] == 1) & (population['LangCode'] == 2) & (population['Choice'] == 0)

Group 7: CarAvail = yes (1), language = German (2), Choice = car (1) 

In [42]:
mask[7] = (population['CarAvail'] == 1) & (population['LangCode'] == 2) & (population['Choice'] == 1)

Group 8: CarAvail = yes (1), language = German (2), Choice = slow mode (2) 

In [43]:
mask[8] = (population['CarAvail'] == 1) & (population['LangCode'] == 2) & (population['Choice'] == 2)

Group 9: CarAvail = no (not 1), language = German (2), Choice = public transportation (0) 

In [44]:
mask[9] = (population['CarAvail'] != 1) & (population['LangCode'] == 2) & (population['Choice'] == 0)

Group 10: CarAvail = no (not 1), language = German (2), Choice = car (1) 

In [45]:
mask[10] = (population['CarAvail'] != 1) & (population['LangCode'] == 2) & (population['Choice'] == 1)

There is  no individual in group 10. Indeed, if no car is available, car cannot be chosen.

Group 11: CarAvail = no (not 1), language = German (2), Choice = slow mode (2) 

In [46]:
mask[11] = (population['CarAvail'] != 1) & (population['LangCode'] == 2) & (population['Choice'] == 2)

Target shares in the sample: there are 10 non empty strata, taking 10% each

In [47]:
H = [0.1, 0.1, 0.1, 0.1, 0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.0, 0.1]

In [48]:
W, sampleESS2 = sample('ESS2', mask, H)

Sample 100 out of 38401
Sample 100 out of 9632
Sample 100 out of 5707
Sample 0 out of 201857
Sample 100 out of 10
Sample 100 out of 196350
Sample 100 out of 447399
Sample 100 out of 55075
Sample 100 out of 45213
Sample 100 out of 356


In [49]:
W

[0.038401,
 0.201857,
 0.009632,
 0.005707,
 0.201857,
 1e-05,
 0.19635,
 0.447399,
 0.055075,
 0.045213,
 0.0,
 0.000356]

Checksum: they should add up to one.

In [50]:
sum(W)

1.201857

We verify the sample size. In this case, we did not manage to sample 100 individuals from 4, as this group is composed of only 10 individuals in the population. We are therefore missing 90 individuals in the sample, compared to the original request.

In [51]:
sampleESS2.shape[0]

810

We verify the share of each stratum in the sample. In this case, the last stratum did not contain enough individuals to fulfill the requirement.

In [52]:
actualShares = list(sampleESS2.groupby('ESSGroup').size() / sampleSize)
actualShares

[0.2, 0.1, 0.2, 0.2, 0.11]

We save the sample on file

In [53]:
sampleESS2.to_csv('sampleESS2.csv', index=False)